In [47]:
!pip install langchain langgraph langchain-tavily langgraph-checkpoint-sqlite langchain-anthropic langchain[google-genai] pydantic

In [2]:
!export LANGSMITH_TRACING="true"
!export LANGSMITH_API_KEY="..."
!export LANGSMITH_PROJECT="default" # or any other project name

In [78]:
import getpass
import os
from google.colab import userdata

# if not os.environ.get("GOOGLE_API_KEY"):
#   os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

In [65]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver


prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

def call_model(state: MessagesState):
  resp = model.invoke(state["messages"])
  return {"messages": resp}


workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

config_user1 = {"configurable": {"thread_id": "abc123"}}
config_user2 = {"configurable": {"thread_id": "abc456"}}

query = "Hi! I'm Jim."
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config_user1)
output["messages"][-1].pretty_print()

input_messages = [HumanMessage("What's my name?")]
output = app.invoke({"messages": input_messages}, config_user1)
output["messages"][-1].pretty_print()

input_messages = [HumanMessage("What's my name?")]
output = app.invoke({"messages": input_messages}, config_user2)
output["messages"][-1].pretty_print()

input_messages = [HumanMessage("What's my name?")]
output = app.invoke({"messages": input_messages}, config_user1)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi Jim! It's nice to meet you.

I'm an AI, so I don't have a name, but you can just call me whatever works for you, or think of me as your helpful assistant.

How can I help you today?
================================== Ai Message ==================================

Your name is Jim! You told me that just a moment ago.

How can I help you, Jim?
================================== Ai Message ==================================

I don't know your name. As an AI, I don't have access to personal information about you or previous interactions that would tell me who you are.
================================== Ai Message ==================================

Your name is Jim.

You've asked me that a couple of times now! Is everything alright, Jim? How can I assist you further?


In [70]:
from langchain_tavily import TavilySearch
import pprint

search = TavilySearch(max_results=2)
# search_results = search.invoke("What is the weather in RETFORD")
# print(search_results)

# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

model_with_tools = model.bind_tools(tools)

def call_model_with_tools(state: MessagesState):
  resp = model_with_tools.invoke(state["messages"])
  return {"messages": resp}

def print_messages(state: MessagesState):
  for msg in state["messages"]:
    msg.pretty_print()
  return state

workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model_with_tools")
workflow.add_node("model_with_tools", call_model_with_tools)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [33]:
query = 'What is the weather in RETFORD(UK)?'
input_messages = [HumanMessage(query)]
# response = app.invoke({"messages": input_messages}, config_user1)
response = model_with_tools.invoke([{"role": "user", "content": query}])
# print_messages(response)
print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")



Message content: 

Tool calls: [{'name': 'tavily_search', 'args': {'query': 'weather in Retford, UK'}, 'id': '66bcc256-1623-4e9d-84c7-caf566cfeaaf', 'type': 'tool_call'}]


In [74]:
from typing import Optional, Union
from typing_extensions import Annotated


# Pydantic
class Weather(TypedDict):
    """How to report weather information to the user."""

    temperature: str = Field(description="The temperature")
    wind: Optional[str] = Field(default=None, description="The wind direction and speed")
    condition: Optional[str] = Field(default=None, description="The full text information")

class ConversationalResponse(TypedDict):
    """Respond in a conversational manner. Be kind and helpful."""

    response: Annotated[str, ..., "A conversational response to the user's query"]

class FinalResponse(TypedDict):
    final_output: Union[Weather, ConversationalResponse]

In [77]:
from langgraph.prebuilt import create_react_agent
from langchain_tavily import TavilySearch

search = TavilySearch(max_results=2)
tools = [search]

model_with_tools = model.bind_tools(tools)

structured_model = model_with_tools.with_structured_output(FinalResponse)

agent_executor = create_react_agent(structured_model, tools)

def call_agent_executor(input_message):
  response = agent_executor.invoke({"messages": [input_message]})

  for message in response["messages"]:
      message.pretty_print()

def call_agent_executor_stream(input_message):
  for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()

ValueError: Missing tools '{'tavily_search'}' in the model.bind_tools()

In [46]:
# call_agent_executor('Hi')
# call_agent_executor("What's the weather like in RETFORD UK?")

call_agent_executor_stream("What's the weather like in RETFORD UK?")

================================ Human Message =================================

What's the weather like in RETFORD UK?
================================== Ai Message ==================================
Tool Calls:
  tavily_search (ad0d10f4-13d5-4b52-b88c-5aa6c816e42b)
 Call ID: ad0d10f4-13d5-4b52-b88c-5aa6c816e42b
  Args:
    query: weather in RETFORD UK
================================= Tool Message =================================
Name: tavily_search

{"query": "weather in RETFORD UK", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Retford, UK", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Retford', 'region': 'Nottinghamshire', 'country': 'United Kingdom', 'lat': 53.3167, 'lon': -0.9432, 'tz_id': 'Europe/London', 'localtime_epoch': 1754838763, 'localtime': '2025-08-10 16:12'}, 'current': {'last_updated_epoch': 1754838000, 'last_updated': '2025-08-10 16:00', 'temp_c': 23.2, 'temp_f': 73.8, 'is_day': 1, 'conditi